In [2]:
!pip install nltk

  Running setup.py install for nltk: started
    Running setup.py install for nltk: finished with status 'done'


In [4]:
!pip install beautifulsoup4

In [5]:
!pip install lxml

In [10]:
!pip install xlrd

In [28]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [1]:
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re
import scipy as s

otto=pd.read_excel('leuphana_datensatz.xlsx')
otto.head(7)

,searchterm,product_id,product_name,product_description,product_sellingpoints
0,waschmaschine,360700745,"Privileg Waschmaschine Toplader PWT 3525, A+, ...",Die PWT 3525 ist eine Toplader-Waschmaschine v...,"<sellingPoints xmlns:rp=""http://www.otto.de/sc..."
1,vans,555220815,Vans T-Shirt,Das T-Shirt von Vans begeistert mit seinem aut...,"<sellingPoints xmlns:rp=""http://www.otto.de/sc..."
2,schreibtisch,403881979,Schreibtisch »8048«,Schreibtisch mit ergonomisch geformter Tischpl...,"<sellingPoints xmlns:rp=""http://www.otto.de/sc..."
3,trampolin,495982357,"4Uniq Trampolin, Ø 305 cm, inkl. Sicherheitsne...","4Uniq Trampolin, Ø 305 cm, inkl. Sicherheitsne...","<sellingPoints xmlns:rp=""http://www.otto.de/sc..."
4,spiegelschrank,536509117,"Spiegelschrank »Junior« Breite 31,5 cm",<p><b>Produktdetails</b></p><ul><li>Serienname...,"<sellingPoints xmlns:rp=""http://www.otto.de/sc..."
5,sekretär,495873227,Loberon Tischsekretär »Lovell«,Tischsekretär Lovell<br> <br> Der hölzerne Tis...,"<sellingPoints xmlns:rp=""http://www.otto.de/sc..."
6,schreibtisch,377847434,Wimex Schreibtisch,Modernes Jugendzimmerprogramm. Pflegeleichte K...,"<sellingPoints xmlns:rp=""http://www.otto.de/sc..."


In [2]:

############Function to clean data

def clean (x):    
    #change to lower case
    x=x.str.lower()

    #Remove nulls
    x = x[~pd.isnull(x)]
    x.index=range(len(x))


    #remove punctuation
    z = pd.Series(s.zeros(len(x)))
    for i in range(len(x)):
        z[i] = re.sub(r'[?|$|*|:|>|<|=|,|\|/|"|(|)|.|+|»|«|!]',r'',x[i])

    #tokenize
    y = pd.Series(s.zeros(len(z)))
    for i in range(len(z)):
        y[i] = word_tokenize(z[i])
       
    #remove words less than 2 characters
    for j in range(len(y)):
        for i in range(len(y[j])):
            if len(y[j][i])<3:
                y[j][i]=""
        
    #remove stop and exclude words
    exclude_words= set(('ja','nein','cm','mehr','oben','','bitte','sellingpoints'))
    stop = set(stopwords.words("german"))
    stops=stop|exclude_words

    w= pd.Series(s.zeros(len(y)))
    for j in range(len(y)):
        w[j]=[i for i in y[j] if i not in stops]
    return w

In [3]:
#########Keywords by search term
setterm='ski'
#change to lower case
otto.searchterm=otto.searchterm.str.lower()

In [4]:
#clean description and product name data
clean_description=clean(otto[otto.searchterm==setterm]['product_description'])
clean_product_name=clean(otto[otto.searchterm==setterm]['product_name'])


In [5]:
#clean selling points data

sellp=otto[otto.searchterm==setterm]['product_sellingpoints']

join_sellp = sellp.str.cat(sep=',')
soup_sellp=BeautifulSoup(join_sellp, "lxml")

text_selling_points=(soup_sellp.text)
selling_points= pd.Series(s.zeros(1))
selling_points[0]=text_selling_points


clean_selling_point=clean(selling_points)


In [6]:
#join outputs
join_descriptions = clean_description.apply(pd.Series).stack().reset_index(drop=True)

join_product_name = clean_product_name.apply(pd.Series).stack().reset_index(drop=True)

join_selling_point = clean_selling_point.apply(pd.Series).stack().reset_index(drop=True)

join_total_output = pd.concat([join_descriptions,join_product_name,join_selling_point])


In [7]:
join_descriptions.head(5)

0                neue
1    allmountain-helm
2               sorgt
3           perfekten
4                halt
dtype: object

In [8]:
join_product_name.head(5)

0             uvex
1              ski
2    snowboardhelm
3             jimm
4             uvex
dtype: object

In [9]:
join_total_output.head(5)

0                neue
1    allmountain-helm
2               sorgt
3           perfekten
4                halt
dtype: object

In [10]:
#Show results by column
results_description = pd.crosstab(join_descriptions, columns="count")  
results_description.sort_values(by=['count'], ascending=False).head(10)


col_0,count
row_0,
100,57
lego®,45
ski-,45
uvex,43
helm,37
material,36
ski,31
einsatzzweck,28
konstruktion,27


In [11]:
#Show results by column


results_product = pd.crosstab(join_product_name, columns="count")  
results_product.sort_values(by=['count'], ascending=False).head(10)


col_0,count
row_0,
ski,60
snowboardhelm,28
uvex,21
trigema,12
alpina,9
skisport,8
langarm,8
lego,7
tec,7


In [12]:
#Show results by column

results_selling_points = pd.crosstab(join_selling_point, columns="count")  
results_selling_points.sort_values(by=['count'], ascending=False).head(10)



col_0,count
row_0,
polyester,9
lego®,7
hautfreundlichsehr,6
sound,6
2016abnehmbare,5
baumwolle,5
waschbare,5
pflegeleichtmodelljahr,4
2016wechselbare,4


In [14]:
results_selling_points 

col_0,count
row_0,
100,2
10000vorgeformte,1
100g,3
10169108,2
120g,3
140gkapuze,1
180°-sichtfeldinnenschale,1
200g,1
2016abnehmbare,5


In [13]:
# Results total output
results = pd.crosstab(join_total_output, columns="count")  
results.sort_values(by=['count'], ascending=False).head(10)

col_0,count
row_0,
ski,95
uvex,64
100,59
lego®,59
ski-,47
helm,37
material,36
snowboardhelm,30
polyester,28
